In [2]:
import ee
import geemap

# Autenticación e inicialización de Google Earth Engine
ee.Authenticate()
ee.Initialize(project="climatech-426614")

# Definir las fechas para antes y después del evento
before_start = '2024-03-30'
before_end = '2024-04-06'
after_start = '2024-04-07'
after_end = '2024-04-20'

# Cargar y configurar la región geográfica de Tanzania
districts = ['Dar-es-salaam'] # 'Dodoma', 'Rufiji District', 'Kibiti District'
admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
tanzania_regions = admin2.filter(ee.Filter.inList('ADM1_NAME', districts))
geometry = tanzania_regions.geometry()

# Crear y configurar el mapa
Map = geemap.Map()
Map.addLayer(geometry, {'color': 'grey'}, 'Affected Regions')

# Configurar y filtrar la colección de imágenes
collection = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .filter(ee.Filter.eq('resolution_meters', 10))\
    .filter(ee.Filter.bounds(geometry))\
    .select('VH')

# Filtrar colecciones para antes y después de las fechas especificadas
before_collection = collection.filter(ee.Filter.date(before_start, before_end))
after_collection = collection.filter(ee.Filter.date(after_start, after_end))

# Mosaico y recorte de imágenes
before = before_collection.mosaic().clip(geometry)
after = after_collection.mosaic().clip(geometry)

Map.centerObject(geometry, 10)
Map.addLayer(before, {'min': -25, 'max': 0}, 'Before Floods', False)
Map.addLayer(after, {'min': -25, 'max': 0}, 'After Floods', False)

# Calculo de diferencias para identificar áreas potencialmente inundadas
difference = after.divide(before)
diff_threshold = 1.25
flooded = difference.gt(diff_threshold).selfMask()
Map.addLayer(flooded, {'palette': 'orange'}, 'Initial Flooded Areas')

# Mostrar el mapa
Map.show()


EEException: Image.gt: If one image has no bands, the other must also have no bands. Got 0 and 1.